In [ ]:
!pip install peft --quiet

In [ ]:
import os
from huggingface_hub import login
login()

# Data Preparation and Tokenization

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, LlamaForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
data = pd.read_csv('/kaggle/input/medquad/medquad.csv')
data.drop(columns=['source'],inplace=True)

# Lets define a fuction to combine the question answer and focus area column
def conc(data):
    return f"Queston: {data['question']} Context: {data['focus_area']} Answer: {data['answer']}"
    
data['text'] = data.apply(conc,axis=1)
data.head()

### Tokenizing the Data

In [ ]:
# initializing the model and the tokenizer
model_name = 'meta-llama/Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)
type(tokenizer)

In [ ]:
dataset = Dataset.from_pandas(data[['text']])

In [ ]:
def tokenize(data):
    # Tokenize and return input_ids and attention_mask
    outputs = tokenizer(
        data['text'],
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors='pt'
    )
    outputs['labels'] = outputs['input_ids'].clone()
    return outputs

tokenizer.pad_token = tokenizer.eos_token # The orignal tokenizer does not have padding defined so we replace it with eos token
# Now we will be mapping the dataset from text --> Embedding
tokenized_dataset = dataset.map(tokenize, batched=True,remove_columns=dataset.column_names)
tokenized_dataset = tokenized_dataset.with_format("torch")
tokenized_dataset

## Setting up the Training

In [ ]:
# Configuring the PEFT parameters
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=2,
    lora_alpha=4,
    lora_dropout=0.1,
    target_modules=['k_proj', 'q_proj', 'v_proj']
)
model = get_peft_model(model, peft_config)

In [ ]:
# Training configuration
total_samples = len(tokenized_dataset)  # Total number of training samples
total_steps = total_samples // 1  # Since per_device_train_batch_size=1


training_args = TrainingArguments(
    output_dir='/kaggle/working/',  # Ensure you have only one output_dir defined
    learning_rate=3e-4,
    per_device_train_batch_size=1,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=total_steps,
    save_strategy='epoch', 
    save_total_limit=2,
    save_only_model=True, 
    push_to_hub=False, 
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Training the Model
trainer.train()

In [ ]:
import zipfile
import os
def zip_dir(directory_path, zip_name):
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, directory_path)
                zipf.write(file_path, arcname)


In [ ]:
zip_dir('/kaggle/working/checkpoint-65648/', '/kaggle/working/checkpoint-65648.zip')
zip_dir('/kaggle/working/checkpoint-49236/', '/kaggle/working/checkpoint-49236.zip')
zip_dir('/kaggle/working/fine_tuned_llama_3b_Medical/', '/kaggle/working/fine_tuned_llama_3b_Medical.zip')

In [ ]:
from transformers import LlamaForCausalLM, AutoTokenizer

base_model_name = "meta-llama/Llama-3.2-3B"
model = LlamaForCausalLM.from_pretrained(base_model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

from peft import PeftModel
model = PeftModel.from_pretrained(model, "/kaggle/input/fine-tune-model/")

# Loading the FineTuned Model

In [ ]:
import os
import torch
from transformers import AutoTokenizer, LlamaForCausalLM
from peft import PeftModel

base_model_name = "meta-llama/Llama-3.2-3B" 
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model_name)
adapter_model_path = "/kaggle/input/fine-tune-model/"
offload_dir = "/kaggle/temp_offload" 
os.makedirs(offload_dir, exist_ok=True)
model = PeftModel.from_pretrained(model, adapter_model_path, offload_dir=offload_dir)


In [ ]:
# Prepare the prompts
system_prompt = "You are a knowledgeable assistant specializing in Biology"
user_prompt = "How to diagnose Parasites - Toxocariasis (also known as Roundworm Infection)?"

# Combine system and user prompts
combined_prompt = f"{system_prompt}\nUser: {user_prompt}\nAssistant:"

# Encode the combined prompt
input_ids = tokenizer.encode(combined_prompt, return_tensors="pt").to(model.device)

# Generate text with adjusted parameters
output = model.generate(
    input_ids,
    max_length=128,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.2, 
    top_k=30,     
    top_p=0.95,      
    repetition_penalty=5.0
)

# Decode and print the output
print(tokenizer.decode(output[0], skip_special_tokens=True))
